In [4]:
# Directories
import os
os.chdir(r"E:\academy\OneDrive - IIT Delhi\3. IIT DELHI\2. Research\JRF\1_PAPER\2_Analysis")
print(os.getcwd())


E:\academy\OneDrive - IIT Delhi\3. IIT DELHI\2. Research\JRF\1_PAPER\2_Analysis


In [5]:
#Array packages
import cartopy.crs as ccrs
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc4

from scipy.stats import kendalltau
#import pymannkendall as mk

#plots
import matplotlib.pyplot as plt
import rioxarray as rio
import geopandas as gpd
from shapely.geometry import mapping
import matplotlib.ticker as ticker

#Progress meter
from dask.diagnostics import ProgressBar
from tqdm import tqdm

# Directories
import os
import glob
import dask
#import h5netcdf
import scipy


# 1. GRIDWISE

### 1. ALL

In [6]:
mdl=5
model=["LIS_MERRA2","LIS_IMD","LIS_CHIRPS","LIS_ERA5","LIS_IMDc","LIS_IMDcnew"]
lsm_M = xr.open_mfdataset(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_grid_All.nc',engine='netcdf4', parallel=True,chunks={'time': 'auto'})


fig, axs = plt.subplots(1,2,subplot_kw={'projection': ccrs.PlateCarree()},figsize=(14,14*1.2),constrained_layout=False)
A=lsm_M["RE_cleaned"].mean(dim="time").compute()
B=lsm_M["RE_cleaned"].std(dim="time").compute()/lsm_M["RE_cleaned"].mean(dim="time").compute()

Map_plot1(A,fig,axs[0],model[mdl],"Average annual RE")
Map_plot1(B,fig,axs[1],model[mdl],"Coefficient of variation RE")
plt.savefig(f"2_RE analysis/1_mean_var/outputs/mn_vr_{model[mdl]}.jpeg", bbox_inches='tight')
plt.show()


OSError: [Errno -51] NetCDF: Unknown file format: b'E:\\academy\\OneDrive - IIT Delhi\\3. IIT DELHI\\2. Research\\JRF\\1_PAPER\\2_Analysis\\DATA_p\\LIS\\LIS_IMDcnew\\LIS_IMDcnew_grid_All.nc'

## 2_REGIONAL

In [15]:
import pymannkendall as mk
from sklearn.linear_model import LinearRegression
from matplotlib.lines import Line2D
import scienceplots

mdl=5
ss=1
model=["LIS_MERRA2","LIS_IMD","LIS_CHIRPS","LIS_ERA5","LIS_IMDc","LIS_IMDcnew"]
var=['TotalPrecip_tavg','Evap_tavg','Qs_tavg','RE','RE_cleaned']
ssn=['Pre-monsoon','Monsoon','Post-monsoon','Winter']
ssn_m=[[3,4,5],[6,7,8,9],[10,11],[12,1,2]]

ds_RM = xr.open_mfdataset(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_All.nc',engine='netcdf4', parallel=True,chunks={'time': 'auto'})
basin=ds_RM.region.values

i=1
s=ds_RM.isel(region=i)
basin


array(['Indus', 'Ganga', 'WFKS', 'Sabarmati', 'Mahi', 'Tapi', 'Narmada',
       'Brahmaputra', 'Subarnarekha', 'Brahmani', 'Mahanadi', 'Godavari',
       'Krishna', 'Pennar', 'Cauvery', 'EFPK', 'EFMP', 'West Flowing'],
      dtype=object)

In [24]:
var = ['TotalPrecip_tavg', 'Evap_tavg', 'Qs_tavg', 'RE', 'RE_cleaned']

fig, ax1 = plt.subplots(1, 1, figsize=(16, 8), facecolor='white', dpi=600, sharey=True, sharex=True)

ax2 = ax1.twinx()

plt.style.use(['science', 'no-latex', 'bright'])

ds_RM1 = s
yr = ds_RM1.time
re = ds_RM1['RE_cleaned']
ppt = ds_RM1['TotalPrecip_tavg']

[x.set_linewidth(1.25) for x in ax1.spines.values()]
ax1.grid(True, color='grey', alpha=0.4, linestyle='dotted')

lw = 1.2; al = 1

# Plot Runoff Efficiency (RE) on ax1
line1 = ax1.plot(yr, re, color='black', label='Runoff Efficiency', alpha=1, linewidth=lw*1.2)

# Plot Precipitation (PPT) on ax2
line2 = ax2.plot(yr, ppt, color='darkblue', label='Precipitation', alpha=1, linewidth=0.9)

# Set ticks and limits for both axes
ax1.xaxis.set_tick_params(which='both', bottom=True, top=False, labelsize=22)
ax1.yaxis.set_tick_params(which='both', left=True, right=False, labelsize=22)
ax1.set_ylim(0, 0.15)

ax2.xaxis.set_tick_params(which='both', bottom=True, top=False, labelsize=22)
ax2.yaxis.set_tick_params(which='both', left=True, right=False, labelsize=22)
ax2.set_ylim(0, 400)

ax1.set_xlim(yr[200], yr[300])

# Add text to the plot
re_var = re.std().compute().values
ppt_var = ppt.std().compute().values

ax1.text(0.5, 0.97, f'{basin[i]}', fontsize=36, weight='bold', ha='left', va='top', transform=ax1.transAxes)

# Adding legends for both ax1 and ax2
lines = line1 + line2  # Combine both lines
labels = [l.get_label() for l in lines]  # Get labels from both axes
ax1.legend(lines, labels, loc='upper right', fontsize=22)  # Place the legend on ax1

# Common labels
fig.text(0.5, -0.02, 'Year', ha='center', fontsize=34, fontweight='bold')
fig.text(-0.02, 0.5, 'Runoff efficiency', va='center', rotation='vertical', fontsize=34, fontweight='bold')
fig.text(1.02, 0.5, 'Precipitation', va='center', rotation='vertical', fontsize=34, fontweight='bold')

# Tight layout and save the figure
plt.tight_layout()
plt.savefig(f'2_RE analysis/5_drought_flood/Ganga.png', dpi=600)
plt.show()


# FUNCTIONS

In [5]:
## Single map function
def Map_plot1(xrr1,fig,axs,plt_tit,cbar_tit):
    import numpy as np 
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import geopandas as gpd
    import matplotlib
    import xarray as xr
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.colors as clr
    import scienceplots
    
    plt.style.use(['science','no-latex','bright'])
    vmin1=xrr1.quantile(0.05)
    vmax1=xrr1.quantile(0.9)
    cmap1='viridis_r'
    
# Step 3 : Figure axis confiureations   
    fig.subplots_adjust(bottom=None, left=None, top=None, right=None, wspace=0.2, hspace=None) 
    [x.set_linewidth(1.5) for x in axs.spines.values()]
    axs.set_extent([67.90, 97,5, 37.50], ccrs.PlateCarree())

    fig.subplots_adjust(bottom=None, left=None, top=None, right=None, wspace=0.15, hspace=None)  

# Step 5 : Plotting the data
    basin_map = gpd.read_file('DATA_p/Shapefiles/Indian_basin/India_transboundary.shp')
    #ind_map = gpd.read_file('DATA/Shapefiles/Indian_map/India_country.shp')
    Main_plot=xrr1.plot(vmin=vmin1,vmax=vmax1,ax=axs,cmap=cmap1,add_labels=False,add_colorbar=False) 
    basin_map.plot(ax=axs,color = 'none',edgecolor = 'white',linewidth=.4,linestyle='--')
    
    axs.set_title(plt_tit,fontsize=15,pad=8,fontname='Times New Roman',weight='bold')
    
# Step 4: Colourbar settings   

    #cbar_ax = fig.add_axes([lf+wd*0.51, bt + 0.026, wd-wd*0.6, 0.005])
    cbar=plt.colorbar(Main_plot, orientation="vertical",shrink=0.28, pad=.045, aspect=20, label = cbar_tit) #,label = cbar_tit
    cbar.outline.set_linewidth(1.1) 
    cbar.ax.tick_params(axis='both',which='both', labelsize=10,width=0.8, length=6) 
    cbar.ax.tick_params(axis='both',which='minor', labelsize=10,width=0.3, length=3)
    cbar.ax.tick_params(axis='both', labelsize=13) 

    cbar.ax.yaxis.labelpad = 10
    cbar.ax.yaxis.label.set_fontsize(16)
    cbar.ax.yaxis.label.set_fontweight('bold')
    cbar.ax.yaxis.label.set_fontname('Times New Roman')

    #cbar.set_label(cbar_tit, fontsize=wd*45, labelpad=-wd*130,weight="bold")#position=(0.5, -0.5)
    
    cbar.locator = ticker.MaxNLocator(nbins=6)  # You can adjust the number of bins as needed
    cbar.update_ticks()

In [6]:
def Map_plot2(xrr,col_row,plt_tit,cbar_tit):

    import numpy as np 
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import geopandas as gpd
    import matplotlib
    import xarray as xr
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.colors as clr
    import scienceplots
    c=col_row[1]
    r=col_row[0]
    # Define the parameters
    B=5   # THis is the base scale. one figure dimension. All are determined based on this

    bt = 1.5  # bottom space
    
    fig_len=c*5 
    fig_hg=r*5.2 +bt   #This 1.15 depend on the subplot dimension. This case, height of subplot more than length
    fnt = B*3.2
    pd = B*1.5
    ln_wd = B*0.2
    
    fig, axs = plt.subplots(col_row[0],col_row[1],subplot_kw={'projection': ccrs.PlateCarree()},figsize=(fig_len,fig_hg),constrained_layout=False)
    axs=axs.flatten()

    #Adding feature maps
    basin_map = gpd.read_file('DATA_p/Shapefiles/Indian_basin/India_transboundary.shp')
    ind_map = gpd.read_file('DATA_p/Shapefiles/Indian_map/India_country.shp')

    #define plot variables
    vmin1 =xrr[0].quantile(0.05)
    vmax1=xrr[0].quantile(0.9)
    #vmin1 =0
    #vmax1=0.016
    cmap2='RdBu'

    #Loop over all of the models
    for i,model in enumerate(range(len(xrr))):

            # Axis setttings
            data=xrr[i]

            axs[i].set_extent([67.90, 97.59,4.5, 37.50], ccrs.PlateCarree())

            axs[i].set_xticks([72,80,88], crs=ccrs.PlateCarree())
            axs[i].set_yticks([10,20,30], crs=ccrs.PlateCarree())
            axs[i].xaxis.set_ticks_position('bottom')
            axs[i].yaxis.set_ticks_position('left')
            axs[i].tick_params(axis='both', labelrotation=90,pad=1,labelsize=7,width=1.2, length=6)
            axs[i].set_xticklabels([])
            axs[i].set_yticklabels([])
            [x.set_linewidth(2) for x in axs[i].spines.values()]        #bOUNDARY WIDTHH 

            #cmap1 = clr.LinearSegmentedColormap.from_list('blue_to_white', ['lightgrey', '#FFFFFF'], N=256)
            #gradient = np.linspace(0, 1, 256).reshape(-1, 1)
            #axs[i].imshow(gradient, cmap=cmap1, extent=[67.90, 97.59,4.5, 37.50], transform=ccrs.PlateCarree())    

            axs[i].set_title(f"{plt_tit[i]}",fontsize=fnt,pad=pd,fontname='Times New Roman',weight='bold')

            # Customize gridlines (NOTE CARTOPY WONT PROVIDE TICKS, SO WE MANUALLY PROVIDED IN ABOVE LINE)
            gl = axs[i].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=0.05, color='black', alpha=0.2, linestyle='-')

            gl.xlabel_style = {"size" : 13,'family': 'Times New Roman','weight': 'normal','rotation': 0}
            gl.ylabel_style = {"size" : 13,'family': 'Times New Roman','weight': 'normal','rotation': 90}
            gl.top_labels = False
            gl.right_labels = False
            gl.xlocator = plt.MaxNLocator(4)  # Adjust the number of x-axis gridlines as needed
            gl.ylocator = plt.MaxNLocator(4) 


            # Adding Mainplo
            Main_plot=data.plot(vmin=vmin1,vmax=vmax1 ,ax=axs[i], cmap=cmap2, add_labels=False, add_colorbar=False) 

            # Adding addition feature 
            basin_map.plot(ax=axs[i],color = 'none',edgecolor = 'black',linewidth=ln_wd,linestyle='--')



    # Adjust the location of the subplots on the page to make room for the colorbar
    fig.subplots_adjust(bottom=1.5/fig_hg, left=None, top=None, right=None, wspace=1.6/fig_len, hspace=2/fig_hg)  # First lock bottom, while make others None (automatically select), Then change spaces

    # Add a colorbar axis at the bottom of the graph
    cbar_ax = fig.add_axes([0.2, 0.63/fig_hg, 0.6, 0.19/fig_hg])   # left,bottom, widht, height

    # Draw the colorbar
    cbar=fig.colorbar(Main_plot, cax=cbar_ax,orientation='horizontal') #,ticks=[0, 1,5]
    cbar.outline.set_linewidth(2) 
    cbar.ax.tick_params(axis='both',which='both', labelsize=fnt,width=2, length=8) 
    cbar.ax.tick_params(axis='both',which='minor', labelsize=fnt*0.7,width=1, length=4) 
    cbar.ax.xaxis.set_ticks_position('top')
    cbar.ax.xaxis.set_label_position('top')

    cbar.set_label(cbar_tit, fontsize=fnt*1.2, labelpad=-55,weight="bold")#position=(0.5, -0.5)
    cbar.ax.xaxis.label.set_fontweight('bold')

    fig.suptitle("", fontsize=19, y=0.915, fontweight='bold', fontname='Times New Roman')
    

In [7]:
## Single map function
def Map_plot3(xrr,col_row,V,plt_tit,cbar_tit):
    import numpy as np 
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import geopandas as gpd
    import matplotlib.pyplot as plt
    import xarray as xr
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.colors as clr
    import matplotlib.ticker as ticker
    import scienceplots
    plt.style.use(['science','no-latex','bright'])
    
    c=col_row[1]
    r=col_row[0]
    B=5
    lt = 0  # bottom space
    fig_len=c*5 + lt
    fig_hg=r*5.3
    fnt = B*3.2
    pd = B*1.5
    ln_wd = B*0.2
    
    fig, axs = plt.subplots(r,c,subplot_kw={'projection': ccrs.PlateCarree()},figsize=(fig_len,fig_hg),constrained_layout=False)
    axs=axs.flatten()

    acron=['a','b','c','d','e','f','g','h']

    for i in range(len(xrr)):    

        Kwarg = {
        'cmap': 'viridis_r',
        'vmin': V[i][0],    # Minimum value for the colormap
        'vmax': V[i][1],     # Maximum value for the colormap
        'alpha': 1,  # Transparency
         }
        
        # Step 3 : Figure axis confiureations   
        fig.subplots_adjust(bottom=None, left=None, top=None, right=0.8, wspace=0.03, hspace=0.13) 
        [x.set_linewidth(1.7) for x in axs[i].spines.values()]
        axs[i].set_extent([67.90, 97,4, 37.50], ccrs.PlateCarree())
    
        lf=axs[i].get_position().x0
        bt=axs[i].get_position().y0
        wd=axs[i].get_position().width
        hg=axs[i].get_position().height
    
        # Step 5 : Plotting the data
        basin_map = gpd.read_file('DATA_p/Shapefiles/Indian_basin/India_transboundary.shp')
        Main_plot=xrr[i].plot(ax=axs[i],**Kwarg,add_labels=False,add_colorbar=False) 
        basin_map.plot(ax=axs[i],color = 'none',edgecolor = 'black',linewidth=.4,linestyle='--')
        
        axs[i].set_title(f"{acron[i]}) {plt_tit[i]}",fontsize=B*4,pad=1.8*B,fontname='Times New Roman',weight='bold')
        
        # Step 4: Colourbar settings   
        if (i+1) % 4 == 0:  # Check if a is a multiple of 4
            cbar_ax = fig.add_axes([lf+wd+0.005, bt+0.02 , 0.008, hg-0.2*hg])
            cbar=plt.colorbar(Main_plot,cax=cbar_ax, orientation="vertical",extend='neither',shrink=0.9, pad=.06, aspect=20) #,label = cbar_tit
            cbar.set_label(cbar_tit[i], fontsize=B*4.2, labelpad=B*3.2)#position=(0.5, -0.5)
            
            cbar.ax.tick_params(axis='both',which='both', labelsize=16,width=0.8, length=5) 
            cbar.ax.tick_params(axis='both',which='minor', labelsize=16,width=0.2, length=3)
            cbar.ax.tick_params(axis='both', labelsize=16) 
            
            cbar.outline.set_linewidth(1) 
            cbar.ax.yaxis.label.set_fontname('Times New Roman')
           # cbar.ax.yaxis.label.set_fontweight('bold')

            cbar.locator = ticker.MaxNLocator(nbins=6)  # You can adjust the number of bins as needed
            cbar.update_ticks()
            
        print(wd)
